Importações iniciais


In [204]:
import ajuste as aj
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)  # Exibe todas as linhas
pd.set_option("display.max_columns", None)  # Exibe todas as colunas (para DataFrames)
e = np.exp(1)


def tal_calcul(df):
    df_reseted = df.reset_index(drop=True)
    valor_procurado = df_reseted["U_b"].iloc[0]/ e #A/e
    
    indice_mais_proximo = (df_reseted["Time"] - valor_procurado).abs().idxmin()
    valor_mais_proximo = df_reseted["Time"].iloc[indice_mais_proximo]
    return (valor_mais_proximo/1000)

def tal_calcul_log(df):
    #offset
    df["U_b"] += 2.5
    df["Time"] /= 1000
    df["Time"] += 0.0025
    df_reseted = df.reset_index(drop=True)
    a = df_reseted["Time"].iloc[0]
    tal = - df_reseted["Time"]/np.log(df_reseted["U_b"]/a)
    return tal


Dados subindo:

In [205]:
path = "RC-01.csv"

dado_sub = aj.slicing_data(path,[2,12],ascending=True) #Dados filtrados

dado_sub_regres, b = aj.regres_exp(dado_sub,[2,12]) #Dados da regressão e o parâmetro b

(aj.data_x_regression(dado_sub,dado_sub_regres)).display() #Gráfico

tal_sb = 1/(b*1000)


alt.LayerChart(...)

Dado descendo

In [ ]:
dado_dsc = aj.slicing_data(path, [1,2.6])

dado_dsc_regres, b = aj.regres_exp(dado_dsc,[1.05,2.6])

(aj.data_x_regression(dado_dsc,dado_dsc_regres)).display()

tal_dc = 1/(b*1000)
tal_carlmethd = tal_calcul(dado_dsc)
tal_log = tal_calcul_log(dado_dsc)


alt.LayerChart(...)

0     -0.000690
1     -0.000691
2     -0.000692
3     -0.000693
4     -0.000695
5     -0.000696
6     -0.000698
7     -0.000699
8     -0.000700
9     -0.000702
10    -0.000703
11    -0.000705
12    -0.000705
13    -0.000707
14    -0.000708
15    -0.000710
16    -0.000711
17    -0.000712
18    -0.000713
19    -0.000715
20    -0.000716
21    -0.000717
22    -0.000719
23    -0.000720
24    -0.000721
25    -0.000722
26    -0.000725
27    -0.000726
28    -0.000727
29    -0.000728
30    -0.000730
31    -0.000731
32    -0.000732
33    -0.000734
34    -0.000735
35    -0.000736
36    -0.000738
37    -0.000739
38    -0.000740
39    -0.000742
40    -0.000743
41    -0.000744
42    -0.000746
43    -0.000747
44    -0.000749
45    -0.000750
46    -0.000752
47    -0.000753
48    -0.000754
49    -0.000756
50    -0.000757
51    -0.000759
52    -0.000761
53    -0.000762
54    -0.000763
55    -0.000764
56    -0.000766
57    -0.000767
58    -0.000768
59    -0.000770
60    -0.000771
61    -0.000772
62    -0

In [207]:
with open("RC-01.txt", "w") as m:

    m.write(f"tal subindo com ajuste: {tal_sb}\n")
    m.write(f"tal_descendo com ajuste: {tal_dc}\n")
    m.write(f"tal_descendo com mt t=tal: {tal_carlmethd}\n")


In [208]:
path = "RC-02.csv"

data_dsc2 = aj.slicing_data(path,[1,3])

data_dsc2_regres, b = aj.regres_exp(data_dsc2,[1,3])

(aj.data_x_regression(data_dsc2,data_dsc2_regres)).display()



tal_dsc2 = 1/(b*1000)
tal_carlmethd = tal_calcul(data_dsc2)


alt.LayerChart(...)

In [209]:
path = "RC-02.csv"

data_sb2 = aj.slicing_data(path,[3,10],ascending=True)

data_sb2_regres, b = aj.regres_exp(data_sb2,[3,10])

(aj.data_x_regression(data_sb2,data_sb2_regres)).display()


tal_sb2 = 1/(b*1000)



alt.LayerChart(...)

In [210]:
with open("RC-02.txt", "w") as m:

    m.write(f"tal subindo com ajuste: {tal_sb2}\n")
    m.write(f"tal_descendo com ajuste: {tal_dsc2}\n")
    m.write(f"tal_descendo com mt t=tal: {tal_carlmethd}\n")
    

print("Arquivo escrito com sucesso!")


Arquivo escrito com sucesso!


In [211]:
import numpy as np
e = np.exp(1)

path = "RC-01.csv"